# Extracting data from the data engineering networking WhatsApp group

In [81]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# Taking a txt file and transforming it into a dataframe

In [82]:
path = "D:/engenharia de dados/files/whats/networking.txt"


with open(path, "r", encoding="utf-8") as f:
    content = f.readlines()

default = re.compile(
    r"^(?P<day>\d{1,2})/\d{1,2}/\d{4} "
    r"(?P<hour>\d{2}:\d{2}) - "
    r"(?:(?P<sender>.*?): )?"
    r"(?P<message>.*)$"
)

data = []
current_message = None

for line in content:
    match = default.match(line)
    if match:
        day = match.group("day")
        hour = match.group("hour")
        sender = match.group("sender") or "SISTEMA"
        message = match.group("message")
        
        current_message = [day, hour, sender if sender else "SISTEMA", message]
        data.append(current_message)
    else:
        if current_message:
            current_message[3] += "\n" + line 

df = pd.DataFrame(data, columns=["Dia", "Hora", "Remetente", "Mensagem"])

print(df[["Dia", "Hora", "Remetente"]])


     Dia   Hora Remetente
0     12  10:09   SISTEMA
1     10  19:08   SISTEMA
2     10  19:08   SISTEMA
3     10  19:08   SISTEMA
4     12  10:09   SISTEMA
...   ..    ...       ...
1216  14  16:42   SISTEMA
1217  14  17:45   SISTEMA
1218  14  19:05   SISTEMA
1219  14  19:31   SISTEMA
1220  14  20:06   SISTEMA

[1221 rows x 3 columns]


# Filtering only messages that contain LinkedIn and transforming the data only into LinkedIn links

In [83]:
df_linkedin = df[df["Mensagem"].str.contains(r"linkedin\.com", case=False, na=False)].copy()

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].str.extract(
    r"((?:https?://)?(?:www\.)?linkedin\.com[^\s]+)",
    expand=False,
    flags=re.IGNORECASE
)

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].apply(
    lambda x: "https://" + x if pd.notnull(x) and not x.startswith("http") else x
)

df_linkedin.rename(columns={"Mensagem": "Linkedin"}, inplace=True)

print(df_linkedin[["Hora", "Linkedin"]])


       Hora                                           Linkedin
168   12:00        https://www.linkedin.com/in/iasmim-horrana/
169   12:00            https://www.linkedin.com/in/max-mitsuya
170   12:00  https://www.linkedin.com/in/jo%C3%A3o-victor-1...
176   12:00  https://www.linkedin.com/in/maria-eduarda-nasc...
178   12:01  https://www.linkedin.com/in/ygor-amaro-114613231/
...     ...                                                ...
1197  13:59  https://www.linkedin.com/in/allanfmachado?utm_...
1198  13:59         https://www.linkedin.com/in/lucaswenceslau
1199  13:59  https://www.linkedin.com/in/riulersilverio?utm...
1200  14:00            https://www.linkedin.com/in/luanbocalon
1201  14:00            https://www.linkedin.com/in/ronneyagra/

[535 rows x 2 columns]


## Removing duplicate data, as they may have sent LinkedIn again on other days

In [84]:
duplicated = df_linkedin[df_linkedin.duplicated()]
duplicated["Linkedin"].head()

779     https://www.linkedin.com/in/miguel-gon%C3%A7al...
1066              https://www.linkedin.com/in/oceanalves/
1154    https://www.linkedin.com/in/maria-eduarda-nasc...
Name: Linkedin, dtype: object

In [88]:
df_linkedin = df_linkedin.drop_duplicates()

# Generating the Excel file, adjusting the column widths, and turning the LinkedIn column into a hyperlink

In [86]:
path_excel = r"D:\\engenharia de dados\\files\\whats\\networking.xlsx"

df_linkedin.to_excel(path_excel, index=False)

wb = load_workbook(path_excel)
ws = wb.active

# Turns Column D of the created Excel file into a hyperlink
for row in range(2, ws.max_row + 1):
    cell = ws[f"D{row}"]
    url = cell.value
    if url:
        cell.hyperlink = url
        cell.value = "Linkedin"
        cell.style = "Hyperlink"

# Adjust the size of the cells to make it visually appealing
for col_cells in ws.columns:
    max_len = 0
    col_letter = get_column_letter(col_cells[0].column)
    for c in col_cells:
        if c.value is not None:
            max_len = max(max_len, len(str(c.value)))
    ws.column_dimensions[col_letter].width = max(10, max_len + 2)

wb.save(path_excel)
